Importing all Libraries

In [2]:
import os
import cv2
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

Load Training Data

In [3]:
#Change the DIRECTORY link according to your file location
DIRECTORY = r'C:\Users\Krishna\Desktop\K.J.TYBTech\NNFL\IA2\flowers'
Categories = ['rose', 'sunflower']

IMAGE_SIZE = 150

data = []

for category in Categories:
    folder = os.path.join(DIRECTORY, category)
    label = Categories.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder,img)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr,(IMAGE_SIZE, IMAGE_SIZE))
        data.append([img_arr,label])

random.shuffle(data)

total_images = []
total_labels = []

train_images = []
train_labels = []

for features, labels in data:
    train_images.append(features)
    train_labels.append(labels)
    total_images.append(features)
    total_labels.append(labels)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

train_images = train_images/255

print(train_images.shape , train_labels.shape)

(1800, 150, 150, 3) (1800,)


Load Testing data

In [4]:
TEST_DIRECTORY = r'C:\Users\Krishna\Desktop\K.J.TYBTech\NNFL\IA2\test-flowers'
Categories = ['rose', 'sunflower']

IMAGE_SIZE = 150
test_data = []

for category in Categories:
    folder = os.path.join(TEST_DIRECTORY, category)
    label = Categories.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder,img)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.resize(img_arr,(IMAGE_SIZE, IMAGE_SIZE))
        # plt.imshow(img_arr)
        # break
        test_data.append([img_arr,label])

random.shuffle(test_data)

test_images = []
test_labels = []

for features, labels in test_data:
    test_images.append(features)
    test_labels.append(labels)

test_images = np.array(test_images)
test_labels = np.array(test_labels)
test_images = test_images/255

print(test_images.shape , test_labels.shape)


(600, 150, 150, 3) (600,)


Splitting data using train_test_split 

In [5]:
from sklearn.model_selection import train_test_split

(trainX, testX, trainY, testY) = train_test_split(total_images, total_labels, test_size=0.15, random_state=42)
trainX = np.array(trainX)
trainX = trainX/255
trainY = np.array(trainY)
testX = np.array(testX)
testX = testX/255
testY = np.array(testY)

Making Tensorboard file for graphs

In [6]:
from keras.callbacks import TensorBoard

NAME = f'flower-prediction-{int(time.time())}'
tens_board = TensorBoard(log_dir=f'logs\\{NAME}\\')

First Model

In [106]:
#Model on CNN Architecture
model = Sequential()

model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Conv2D(64,(3,3), activation= 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten())

model.add(Dense(128,input_shape = train_images.shape[1:], activation='relu'))
model.add(Dense(2,activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_images,train_labels, epochs= 10, validation_split= 0.1, batch_size= 32, callbacks=[tens_board])

In [ ]:
#prediction on test data
print("Evaluate on test data")
predict_result = model.evaluate(test_images, test_labels, batch_size=32)
print("test loss, test acc:", predict_result)

Second Model

In [ ]:
#Load VGG16 Model
from keras.applications.vgg16 import VGG16

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_images[0].shape)
base_model.trainable = False ## Not trainable weights
Vggmodel = Sequential()

Vggmodel.add(base_model)
Vggmodel.add(Flatten())
Vggmodel.add(Dense(50, input_shape = train_images.shape[1:], activation='relu'))
Vggmodel.add(Dense(20, activation='relu'))
Vggmodel.add(Dense(5, activation='softmax'))

Vggmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
Vggmodel.fit(train_images, train_labels, epochs=10, validation_split=0.1, batch_size=32, callbacks=[tens_board])

In [ ]:
#prediction on test data
print("Evaluate on test data")
Vgg_predict_result = Vggmodel.evaluate(test_images, test_labels, batch_size=32)
print("test loss, test acc:", Vgg_predict_result)

Third Model

In [12]:

#Load ResNet50 Model
from keras.applications.resnet import ResNet50

base_model = ResNet50(weights='imagenet',include_top=False, input_shape=(150,150,3))
base_model.trainable = False

ResNetmodel = Sequential()
ResNetmodel.add(base_model)  
ResNetmodel.add(Flatten())
ResNetmodel.add(Dense(50, input_shape = train_images.shape[1:], activation='relu'))
ResNetmodel.add(Dense(20, activation='relu'))
ResNetmodel.add(Dense(5, activation='softmax'))

ResNetmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
ResNetmodel.fit(train_images, train_labels, epochs=10, validation_split=0.1, batch_size=32, callbacks=[tens_board])

Epoch 1/10
51/51 [==============================] - 110s 2s/step - loss: 0.7171 - accuracy: 0.5636 - val_loss: 0.5852 - val_accuracy: 0.7444
Epoch 2/10
51/51 [==============================] - 97s 2s/step - loss: 0.6179 - accuracy: 0.6519 - val_loss: 0.6072 - val_accuracy: 0.6111
Epoch 3/10
51/51 [==============================] - 93s 2s/step - loss: 0.5994 - accuracy: 0.6735 - val_loss: 0.6265 - val_accuracy: 0.6056
Epoch 4/10
51/51 [==============================] - 97s 2s/step - loss: 0.5484 - accuracy: 0.7290 - val_loss: 0.4781 - val_accuracy: 0.8167
Epoch 5/10
51/51 [==============================] - 95s 2s/step - loss: 0.4913 - accuracy: 0.7593 - val_loss: 0.5424 - val_accuracy: 0.6889
Epoch 6/10
51/51 [==============================] - 95s 2s/step - loss: 0.4671 - accuracy: 0.7759 - val_loss: 0.4555 - val_accuracy: 0.8556
Epoch 7/10
51/51 [==============================] - 99s 2s/step - loss: 0.4439 - accuracy: 0.7975 - val_loss: 0.4393 - val_accuracy: 0.8667
Epoch 8/10
51/51 [=

In [13]:
#prediction on test data
print("Evaluate on test data")
ResNet_predict_result = ResNetmodel.evaluate(test_images, test_labels, batch_size=32)
print("test loss, test acc:", ResNet_predict_result)

Evaluate on test data
19/19 [==============================] - 32s 2s/step - loss: 0.8267 - accuracy: 0.5700
test loss, test acc: [0.8266504406929016, 0.5699999928474426]
